# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

Spark está escrito en scala(java), por lo que para usarlo desde python hay una capa de compatibilidad.

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.
No es exactamente como las columnas de panda. Sirve para formar sentencias con SQL.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.
Window fucntions que trabajan en mas de un row.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [2]:
from pyspark.sql import SparkSession

In [4]:
SparkSession.sparkContext

In [5]:
session = SparkSession.builder.getOrCreate()

In [6]:
session.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
session = SparkSession.builder\
                .config('someoption', 'somevalue')\
                .config('anotheroption', 'anothervalue')\
                .getOrCreate()

In [8]:
session.sparkContext.getConf().getAll()

[('spark.driver.host', '10.2.5.243'),
 ('anotheroption', 'anothervalue'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1539963879965'),
 ('spark.serializer.objectStreamReset', '100'),
 ('someoption', 'somevalue'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '49641'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

#### Passing other options to spark session:
    
    

We can check option values in the resulting session like this:

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [9]:
import random

random.choice(['widgeteer', 'smoke salesman', 'wizard', 'psycopath'])


'widgeteer'

In [10]:
random.seed(42)

ids = range(20)
positions = [random.choice(['widgeteer', 'smoke salesman', 'wizard', 'psycopath']) for id in ids]
positions

['widgeteer',
 'widgeteer',
 'wizard',
 'smoke salesman',
 'smoke salesman',
 'smoke salesman',
 'widgeteer',
 'widgeteer',
 'psycopath',
 'widgeteer',
 'widgeteer',
 'widgeteer',
 'smoke salesman',
 'smoke salesman',
 'widgeteer',
 'smoke salesman',
 'psycopath',
 'smoke salesman',
 'psycopath',
 'wizard']

In [11]:
rows = (zip(ids, positions))

In [12]:
df = session.createDataFrame(rows)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
df

DataFrame[_1: bigint, _2: string]

In [15]:
df.take(5)

[Row(_1=0, _2='widgeteer'),
 Row(_1=1, _2='widgeteer'),
 Row(_1=2, _2='wizard'),
 Row(_1=3, _2='smoke salesman'),
 Row(_1=4, _2='smoke salesman')]

In [17]:
df.show(5)

+---+--------------+
| _1|            _2|
+---+--------------+
|  0|     widgeteer|
|  1|     widgeteer|
|  2|        wizard|
|  3|smoke salesman|
|  4|smoke salesman|
+---+--------------+
only showing top 5 rows



In [18]:
from pyspark.sql import Row

help(Row)

Help on class Row in module pyspark.sql.types:

class Row(builtins.tuple)
 |  A row in L{DataFrame}.
 |  The fields in it can be accessed:
 |  
 |  * like attributes (``row.key``)
 |  * like dictionary values (``row[key]``)
 |  
 |  ``key in row`` will search through row keys.
 |  
 |  Row can be used to create a row object by using named arguments,
 |  the fields will be sorted by names. It is not allowed to omit
 |  a named argument to represent the value is None or missing. This should be
 |  explicitly set to None in this case.
 |  
 |  >>> row = Row(name="Alice", age=11)
 |  >>> row
 |  Row(age=11, name='Alice')
 |  >>> row['name'], row['age']
 |  ('Alice', 11)
 |  >>> row.name, row.age
 |  ('Alice', 11)
 |  >>> 'name' in row
 |  True
 |  >>> 'wrong_key' in row
 |  False
 |  
 |  Row also can be used to create another Row like class, then it
 |  could be used to create Row objects, such as
 |  
 |  >>> Person = Row("name", "age")
 |  >>> Person
 |  <Row(name, age)>
 |  >>> 'name' 

In [19]:
row = df.first()

In [20]:
row

Row(_1=0, _2='widgeteer')

first() es una action. Truquillo: Si devuelve algo que no es un dataframe, es una action

In [21]:
row._1

0

In [22]:
row._2

'widgeteer'

In [23]:
row.count

<function Row.count>

In [24]:
row.index

<function Row.index>

In [27]:
rows = zip(ids, positions)
df = session.createDataFrame(rows, schema=['id_number', 'position'])
df.show(5)

+---------+--------------+
|id_number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+
only showing top 5 rows



In [28]:
df.printSchema()

root
 |-- id_number: long (nullable = true)
 |-- position: string (nullable = true)



In [29]:
df.rdd

MapPartitionsRDD[42] at javaToPython at NativeMethodAccessorImpl.java:0

In [30]:
df.rdd.take(5)

[Row(id_number=0, position='widgeteer'),
 Row(id_number=1, position='widgeteer'),
 Row(id_number=2, position='wizard'),
 Row(id_number=3, position='smoke salesman'),
 Row(id_number=4, position='smoke salesman')]

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [40]:
lines = session.sparkContext.textFile('coupon150720.csv')
lines

coupon150720.csv MapPartitionsRDD[53] at textFile at NativeMethodAccessorImpl.java:0

In [32]:
lines.take(2)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0']

In [33]:
def parse (line):
    
    fields = line.split(",")
    coupon = (fields[0], fields[2], fields[3], fields[4], float(fields[6]))
    return coupon



In [34]:
coupons = lines.map(parse)

In [35]:
coupons.take(2)

[('79062005698500', 'MAA', 'AUH', '9W', 56.79),
 ('79062005698500', 'AUH', 'CDG', '9W', 84.34)]

In [ ]:
coupons.

In [38]:
df = session.createDataFrame(lines.)

AssertionError: dataType should be DataType

### Inferring and specifying schemas

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [51]:
from pyspark.sql import types

In [52]:

types.IntegerType()

IntegerType

El schema de un dataframe comprende nombre + tipo + si son nullables o no


In [57]:
employee_schema = types.StructType([types.StructField('id number', types.LongType(), nullable=False),
                 types.StructField('position', types.StringType(), nullable=True)]) #nullable = True es por defencto

In [59]:
employees = session.createDataFrame(zip(ids, positions), schema=employee_schema)

In [60]:
employees.printSchema()

root
 |-- id number: long (nullable = false)
 |-- position: string (nullable = true)



#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [43]:
df_from_csv = session.read.csv('coupon150720.csv')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [47]:
df_from_csv = session.sql('SELECT * from csv. `coupon150720.csv`')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [50]:
df_from_csv = session.sql('SELECT _c0, _c1, _c2, _c3, _c4, _c5, CAST (_c6 AS FLOAT) from csv. `coupon150720.csv`')
df_from_csv.show(5)
df_from_csv.printSchema()

+--------------+---+---+---+---+---+------+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|
+--------------+---+---+---+---+---+------+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|
+--------------+---+---+---+---+---+------+
only showing top 5 rows

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: float (nullable = true)



#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [62]:
session.read.parquet
session.read.json
session.read.jdbc

<bound method DataFrameReader.jdbc of <pyspark.sql.readwriter.DataFrameReader object at 0x112bf6588>>

### Basic operations with DataFrames

In [63]:
employees.show(2)

+---------+---------+
|id number| position|
+---------+---------+
|        0|widgeteer|
|        1|widgeteer|
+---------+---------+
only showing top 2 rows



In [64]:
employee_0 = employees.first()

In [67]:
employee_0['id number']

0

### Filtering and selecting

Syntax inspired in SQL.

In [68]:
employees.select('id number')

DataFrame[id number: bigint]

If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [69]:
type(employees['id number'])

pyspark.sql.column.Column

In [70]:
employees.filter(employees['id number'] <5).show()

+---------+--------------+
|id number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+



In [71]:
employees.where(employees['id number'] <5).show()

+---------+--------------+
|id number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
+---------+--------------+



In [72]:
employees['id number'] <5

Column<b'(id number < 5)'>

In [73]:
(employees['id number'] <5).collect()

TypeError: 'Column' object is not callable

That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

`where` is exactly synonimous with `filter`

A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

#### Exercise

Extract all employee ids which correspond to psycopath

In [79]:
employees.select('id number').where(employees['position'] == 'psycopath').show()

+---------+
|id number|
+---------+
|        8|
|       16|
|       18|
+---------+



In [76]:
employees.filter(employees['position'] == 'psycopath').show()

+---------+---------+
|id number| position|
+---------+---------+
|        8|psycopath|
|       16|psycopath|
|       18|psycopath|
+---------+---------+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [81]:
employees['salary'] = employees['id number'] ** 2

TypeError: 'DataFrame' object does not support item assignment

No se puede hacer esto porque el dataframe es inmutable

In [82]:
employees.withColumn('square', employees['id number'] ** 2)

DataFrame[id number: bigint, position: string, square: double]

In [83]:
employees

DataFrame[id number: bigint, position: string]

In [84]:
employees.select('id number', 
                'position', 
                employees['id number'] ** 2)

DataFrame[id number: bigint, position: string, POWER(id number, 2): double]

In [86]:
employees.select('id number', 
                'position', 
                (employees['id number'] ** 2).alias('squared'))

DataFrame[id number: bigint, position: string, squared: double]

### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [87]:
from pyspark.sql import functions

In [ ]:
#functions.log1p # hace el log(1+n). Se hace para no tener problemas con el log(0)

In [91]:
df = employees.select('id number','position', functions.log1p('id number'))


In [92]:
df.show(2)

+---------+---------+------------------+
|id number| position|  LOG1P(id number)|
+---------+---------+------------------+
|        0|widgeteer|               0.0|
|        1|widgeteer|0.6931471805599453|
+---------+---------+------------------+
only showing top 2 rows



This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

In [93]:
import math

math.log1p(0)

0.0

But we can transform it into a udf:

In [94]:
functions.udf

<function pyspark.sql.functions.udf(f=None, returnType=StringType)>

In [95]:
udf_log1p = functions.udf(math.log1p)

In [97]:
df = employees.select('id number','position', udf_log1p(employees['id number']))

In [98]:
df.show(2)

+---------+---------+------------------+
|id number| position|  log1p(id number)|
+---------+---------+------------------+
|        0|widgeteer|               0.0|
|        1|widgeteer|0.6931471805599453|
+---------+---------+------------------+
only showing top 2 rows



We can do the same with any function we dream up:

In [99]:
shorten = functions.udf(lambda word: word[:3])

In [101]:
df = employees.select('id number','position', shorten(employees['position']).alias('short_pos'))
df.show(5)

+---------+--------------+---------+
|id number|      position|short_pos|
+---------+--------------+---------+
|        0|     widgeteer|      wid|
|        1|     widgeteer|      wid|
|        2|        wizard|      wiz|
|        3|smoke salesman|      smo|
|        4|smoke salesman|      smo|
+---------+--------------+---------+
only showing top 5 rows



Esta udf puede usarse con multiples columnas. Por defecto el udf devuelve un string

If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

Think about this function: what is its return type?

In [105]:
def anonymous(element):
    return element + element

In [106]:
print(anonymous(3))

6


In [107]:
anonymous("pp")

'pppp'

In [109]:
udf_logp1_2 = functions.udf(math.log1p, returnType=types.FloatType())
udf_logp1_2.returnType

FloatType

#### Exercise: 

Create a 'salary' field in our df. make it 100000 for psycopath, 35000 for widgeteer and 50000 for smoke salesman 60000 wizard.



In [112]:
employees.show(10)

+---------+--------------+
|id number|      position|
+---------+--------------+
|        0|     widgeteer|
|        1|     widgeteer|
|        2|        wizard|
|        3|smoke salesman|
|        4|smoke salesman|
|        5|smoke salesman|
|        6|     widgeteer|
|        7|     widgeteer|
|        8|     psycopath|
|        9|     widgeteer|
+---------+--------------+
only showing top 10 rows



In [126]:
def get_salary(position):
    if position == 'widgeteer': return 35000
    elif position == 'wizard': return 60000
    elif position == 'psycopath': return 100000
    elif position == 'smoke salesman': return 50000
    
get_salary_udf = functions.udf(get_salary, returnType=types.IntegerType())

In [127]:
df_with_salary = employees.select('id number', 'position', get_salary_udf(employees['position']).alias('salary'))
df_with_salary.show(5)

+---------+--------------+------+
|id number|      position|salary|
+---------+--------------+------+
|        0|     widgeteer| 35000|
|        1|     widgeteer| 35000|
|        2|        wizard| 60000|
|        3|smoke salesman| 50000|
|        4|smoke salesman| 50000|
+---------+--------------+------+
only showing top 5 rows



In [124]:
def get_salary2(position):
    
    salary_table = {'widgeteer': 35000, 'wizard': 60000, 'psycopath': 100000, 'smoke salesman':  50000}
    
    return salary_table(position)


get_salary2_udf = functions.udf(get_salary2)


If we have a column that is not the desired type, we can convert it with `cast`.

In [129]:
df_with_salary.select('id number',
                     'position',
                     df_with_salary['salary'].cast(types.IntegerType()))

DataFrame[id number: bigint, position: string, salary: int]

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [130]:
df_with_salary.stat.corr('id number', 'salary')

0.38648340657639785

In [131]:
df_with_salary.stat.cov('id number', 'salary')

50789.47368421052

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [132]:
location_udf = functions.udf(lambda: random.choice(['Madrid', 'Barcelona']))

In [135]:
df = df_with_salary.select('id number',
                          'position',
                          'salary',
                          location_udf().alias('location'))

In [145]:
df.show(5)

+---------+--------------+------+---------+
|id number|      position|salary| location|
+---------+--------------+------+---------+
|        0|     widgeteer| 35000|   Madrid|
|        1|     widgeteer| 35000|Barcelona|
|        2|        wizard| 60000|   Madrid|
|        3|smoke salesman| 50000|Barcelona|
|        4|smoke salesman| 50000|   Madrid|
+---------+--------------+------+---------+
only showing top 5 rows



Probar a lanzar varias veces e. df.show().
Se ve que el show es la accion y cada vez la salida es diference

In [141]:
# Para evitar esto, lo cacheamos
df.cache()
df.show()


+---------+--------------+------+---------+
|id number|      position|salary| location|
+---------+--------------+------+---------+
|        0|     widgeteer| 35000|   Madrid|
|        1|     widgeteer| 35000|Barcelona|
|        2|        wizard| 60000|   Madrid|
|        3|smoke salesman| 50000|Barcelona|
|        4|smoke salesman| 50000|   Madrid|
|        5|smoke salesman| 50000|   Madrid|
|        6|     widgeteer| 35000|Barcelona|
|        7|     widgeteer| 35000|   Madrid|
|        8|     psycopath|100000|Barcelona|
|        9|     widgeteer| 35000|   Madrid|
|       10|     widgeteer| 35000|   Madrid|
|       11|     widgeteer| 35000|Barcelona|
|       12|smoke salesman| 50000|   Madrid|
|       13|smoke salesman| 50000|Barcelona|
|       14|     widgeteer| 35000|   Madrid|
|       15|smoke salesman| 50000|   Madrid|
|       16|     psycopath|100000|Barcelona|
|       17|smoke salesman| 50000|Barcelona|
|       18|     psycopath|100000|   Madrid|
|       19|        wizard| 60000

In [146]:
df.crosstab('position', 'location').show()

+-----------------+---------+------+
|position_location|Barcelona|Madrid|
+-----------------+---------+------+
|   smoke salesman|        3|     4|
|           wizard|        0|     2|
|        psycopath|        2|     1|
|        widgeteer|        3|     5|
+-----------------+---------+------+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [148]:
gb = df.groupBy('location')

In [149]:
gb.avg().show()

+---------+-----------------+------------------+
| location|   avg(id number)|       avg(salary)|
+---------+-----------------+------------------+
|   Madrid|9.583333333333334|49583.333333333336|
|Barcelona|            9.375|           56875.0|
+---------+-----------------+------------------+



In [150]:
gb.mean().show()

+---------+-----------------+------------------+
| location|   avg(id number)|       avg(salary)|
+---------+-----------------+------------------+
|   Madrid|9.583333333333334|49583.333333333336|
|Barcelona|            9.375|           56875.0|
+---------+-----------------+------------------+



In [151]:
gb.avg('salary').show()

+---------+------------------+
| location|       avg(salary)|
+---------+------------------+
|   Madrid|49583.333333333336|
|Barcelona|           56875.0|
+---------+------------------+



In [152]:
gb.agg({'id number' : 'max', 'salary': 'mean' }).show()

+---------+--------------+------------------+
| location|max(id number)|       avg(salary)|
+---------+--------------+------------------+
|   Madrid|            19|49583.333333333336|
|Barcelona|            17|           56875.0|
+---------+--------------+------------------+



Esta manera de hacerlo no te deja hacer multiples agg sobre la misma columna

GroupedData has several aggregation functions defined:

We can do several aggregations in a single step, with a number of different syntaxes:

In [158]:
gb.agg(functions.mean('salary').alias('mean salary'),
functions.count('salary'),
functions.countDistinct('position')).show()

+---------+------------------+-------------+------------------------+
| location|       mean salary|count(salary)|count(DISTINCT position)|
+---------+------------------+-------------+------------------------+
|   Madrid|49583.333333333336|           12|                       4|
|Barcelona|           56875.0|            8|                       3|
+---------+------------------+-------------+------------------------+



### Intersections


Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [173]:
random.seed(42)

raises = session.createDataFrame(list(zip([random.choice(['Barcelona', 'Sevilla', 'Madrid']) for _ in range(7)],
                                        [random.randint(0,19) for _ in range(7)],
                                        [random.random() * 10000 for _ in range(7)])),
                                 schema = ['position', 'id number', 'raise']).cache()

In [176]:
raises.show()

+---------+---------+------------------+
| position|id number|             raise|
+---------+---------+------------------+
|   Madrid|        4|297.97219438070346|
|Barcelona|        3|2186.3797480360336|
|Barcelona|       17| 5053.552881033624|
|   Madrid|        2|265.35969683863624|
|  Sevilla|       18| 1988.376506866485|
|Barcelona|       13|6498.8443777952325|
|Barcelona|        1| 5449.414806032167|
+---------+---------+------------------+



In [177]:
df.join(raises, on='id number', how='left').show()

+---------+--------------+------+---------+---------+------------------+
|id number|      position|salary| location| position|             raise|
+---------+--------------+------+---------+---------+------------------+
|        0|     widgeteer| 35000|   Madrid|     null|              null|
|        1|     widgeteer| 35000|Barcelona|Barcelona| 5449.414806032167|
|        2|        wizard| 60000|   Madrid|   Madrid|265.35969683863624|
|        3|smoke salesman| 50000|Barcelona|Barcelona|2186.3797480360336|
|        4|smoke salesman| 50000|   Madrid|   Madrid|297.97219438070346|
|        5|smoke salesman| 50000|   Madrid|     null|              null|
|        6|     widgeteer| 35000|Barcelona|     null|              null|
|        7|     widgeteer| 35000|   Madrid|     null|              null|
|        8|     psycopath|100000|Barcelona|     null|              null|
|        9|     widgeteer| 35000|   Madrid|     null|              null|
|       10|     widgeteer| 35000|   Madrid|     nul

In [183]:
df.join(raises, 
        (df['id number'] == raises['id number']) & 
        (df['location'] == raises['position'])).show()

+---------+--------------+------+---------+---------+---------+------------------+
|id number|      position|salary| location| position|id number|             raise|
+---------+--------------+------+---------+---------+---------+------------------+
|        1|     widgeteer| 35000|Barcelona|Barcelona|        1| 5449.414806032167|
|        2|        wizard| 60000|   Madrid|   Madrid|        2|265.35969683863624|
|        3|smoke salesman| 50000|Barcelona|Barcelona|        3|2186.3797480360336|
|        4|smoke salesman| 50000|   Madrid|   Madrid|        4|297.97219438070346|
|       13|smoke salesman| 50000|Barcelona|Barcelona|       13|6498.8443777952325|
|       17|smoke salesman| 50000|Barcelona|Barcelona|       17| 5053.552881033624|
+---------+--------------+------+---------+---------+---------+------------------+



Observar que el and no vale, hay que usar &

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [184]:
session.sparkContext.uiWebUrl


'http://10.2.5.243:4041'

#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

In [185]:
df.show(5)

+---------+--------------+------+---------+
|id number|      position|salary| location|
+---------+--------------+------+---------+
|        0|     widgeteer| 35000|   Madrid|
|        1|     widgeteer| 35000|Barcelona|
|        2|        wizard| 60000|   Madrid|
|        3|smoke salesman| 50000|Barcelona|
|        4|smoke salesman| 50000|   Madrid|
+---------+--------------+------+---------+
only showing top 5 rows



In [202]:
agg_df = df.groupBy('location')\
            .agg(functions.mean('salary').alias('mean_salary'),
                 functions.stddev('salary').alias('std'))

In [203]:
agg_df.show()

+---------+------------------+------------------+
| location|       mean_salary|               std|
+---------+------------------+------------------+
|   Madrid|49583.333333333336|18642.367842843323|
|Barcelona|           56875.0|  27508.1156855832|
+---------+------------------+------------------+



In [197]:
df_ext = df.join(agg_df, on='location', how='left')

In [200]:
df_ext = df_ext.withColumn('zScore', df_ext['mean_salary'] / df_ext['std'])

In [201]:
df_ext.show()

+---------+---------+--------------+------+------------------+------------------+------------------+
| location|id number|      position|salary|       mean_salary|               std|            zScore|
+---------+---------+--------------+------+------------------+------------------+------------------+
|   Madrid|        0|     widgeteer| 35000|49583.333333333336|18642.367842843323|2.6597122077691453|
|Barcelona|        1|     widgeteer| 35000|           56875.0|  27508.1156855832| 2.067571645040295|
|   Madrid|        2|        wizard| 60000|49583.333333333336|18642.367842843323|2.6597122077691453|
|Barcelona|        3|smoke salesman| 50000|           56875.0|  27508.1156855832| 2.067571645040295|
|   Madrid|        4|smoke salesman| 50000|49583.333333333336|18642.367842843323|2.6597122077691453|
|   Madrid|        5|smoke salesman| 50000|49583.333333333336|18642.367842843323|2.6597122077691453|
|Barcelona|        6|     widgeteer| 35000|           56875.0|  27508.1156855832| 2.0675716

In [205]:
df_ext = df_ext.select('location', 'id number', 'position', 'salary', 'zScore')
df_ext.show(4)
    

+---------+---------+--------------+------+------------------+
| location|id number|      position|salary|            zScore|
+---------+---------+--------------+------+------------------+
|   Madrid|        0|     widgeteer| 35000|2.6597122077691453|
|Barcelona|        1|     widgeteer| 35000| 2.067571645040295|
|   Madrid|        2|        wizard| 60000|2.6597122077691453|
|Barcelona|        3|smoke salesman| 50000| 2.067571645040295|
+---------+---------+--------------+------+------------------+
only showing top 4 rows



1) Calculate the mean and std of salary for each location

2) Annotate each employee with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

In [206]:
df

DataFrame[id number: bigint, position: string, salary: int, location: string]

In [218]:
the_others = session.createDataFrame([(100, 'superboss', 1200000, None),
                                      (101, None, 1000000, 'Miami'),
                                      (102, None, None, None),
                                      (103, None, None, 'Palencia')], 
                                    schema = df.schema)

In [221]:
the_others.show()


+---------+---------+-------+--------+
|id number| position| salary|location|
+---------+---------+-------+--------+
|      100|superboss|1200000|    null|
|      101|     null|1000000|   Miami|
|      102|     null|   null|    null|
|      103|     null|   null|Palencia|
+---------+---------+-------+--------+



In [222]:
df_union = df.union(the_others)
df_union.show(25)

+---------+--------------+-------+---------+
|id number|      position| salary| location|
+---------+--------------+-------+---------+
|        0|     widgeteer|  35000|   Madrid|
|        1|     widgeteer|  35000|Barcelona|
|        2|        wizard|  60000|   Madrid|
|        3|smoke salesman|  50000|Barcelona|
|        4|smoke salesman|  50000|   Madrid|
|        5|smoke salesman|  50000|   Madrid|
|        6|     widgeteer|  35000|Barcelona|
|        7|     widgeteer|  35000|   Madrid|
|        8|     psycopath| 100000|Barcelona|
|        9|     widgeteer|  35000|   Madrid|
|       10|     widgeteer|  35000|   Madrid|
|       11|     widgeteer|  35000|Barcelona|
|       12|smoke salesman|  50000|   Madrid|
|       13|smoke salesman|  50000|Barcelona|
|       14|     widgeteer|  35000|   Madrid|
|       15|smoke salesman|  50000|   Madrid|
|       16|     psycopath| 100000|Barcelona|
|       17|smoke salesman|  50000|Barcelona|
|       18|     psycopath| 100000|   Madrid|
|       19

In [224]:
df_union.dropna(how='all').show(25)

+---------+--------------+-------+---------+
|id number|      position| salary| location|
+---------+--------------+-------+---------+
|        0|     widgeteer|  35000|   Madrid|
|        1|     widgeteer|  35000|Barcelona|
|        2|        wizard|  60000|   Madrid|
|        3|smoke salesman|  50000|Barcelona|
|        4|smoke salesman|  50000|   Madrid|
|        5|smoke salesman|  50000|   Madrid|
|        6|     widgeteer|  35000|Barcelona|
|        7|     widgeteer|  35000|   Madrid|
|        8|     psycopath| 100000|Barcelona|
|        9|     widgeteer|  35000|   Madrid|
|       10|     widgeteer|  35000|   Madrid|
|       11|     widgeteer|  35000|Barcelona|
|       12|smoke salesman|  50000|   Madrid|
|       13|smoke salesman|  50000|Barcelona|
|       14|     widgeteer|  35000|   Madrid|
|       15|smoke salesman|  50000|   Madrid|
|       16|     psycopath| 100000|Barcelona|
|       17|smoke salesman|  50000|Barcelona|
|       18|     psycopath| 100000|   Madrid|
|       19

In [225]:
df_union.dropna(how='all', subset=['salary', 'location']).show(25)

+---------+--------------+-------+---------+
|id number|      position| salary| location|
+---------+--------------+-------+---------+
|        0|     widgeteer|  35000|   Madrid|
|        1|     widgeteer|  35000|Barcelona|
|        2|        wizard|  60000|   Madrid|
|        3|smoke salesman|  50000|Barcelona|
|        4|smoke salesman|  50000|   Madrid|
|        5|smoke salesman|  50000|   Madrid|
|        6|     widgeteer|  35000|Barcelona|
|        7|     widgeteer|  35000|   Madrid|
|        8|     psycopath| 100000|Barcelona|
|        9|     widgeteer|  35000|   Madrid|
|       10|     widgeteer|  35000|   Madrid|
|       11|     widgeteer|  35000|Barcelona|
|       12|smoke salesman|  50000|   Madrid|
|       13|smoke salesman|  50000|Barcelona|
|       14|     widgeteer|  35000|   Madrid|
|       15|smoke salesman|  50000|   Madrid|
|       16|     psycopath| 100000|Barcelona|
|       17|smoke salesman|  50000|Barcelona|
|       18|     psycopath| 100000|   Madrid|
|       19

In [227]:
df_union.dropna(subset=['salary', 'location', 'position'], thresh=2).show(25)
#threshold es el mínimo número de valores no nulos que exigimos para quedarnos

+---------+--------------+-------+---------+
|id number|      position| salary| location|
+---------+--------------+-------+---------+
|        0|     widgeteer|  35000|   Madrid|
|        1|     widgeteer|  35000|Barcelona|
|        2|        wizard|  60000|   Madrid|
|        3|smoke salesman|  50000|Barcelona|
|        4|smoke salesman|  50000|   Madrid|
|        5|smoke salesman|  50000|   Madrid|
|        6|     widgeteer|  35000|Barcelona|
|        7|     widgeteer|  35000|   Madrid|
|        8|     psycopath| 100000|Barcelona|
|        9|     widgeteer|  35000|   Madrid|
|       10|     widgeteer|  35000|   Madrid|
|       11|     widgeteer|  35000|Barcelona|
|       12|smoke salesman|  50000|   Madrid|
|       13|smoke salesman|  50000|Barcelona|
|       14|     widgeteer|  35000|   Madrid|
|       15|smoke salesman|  50000|   Madrid|
|       16|     psycopath| 100000|Barcelona|
|       17|smoke salesman|  50000|Barcelona|
|       18|     psycopath| 100000|   Madrid|
|       19

In [228]:
df_union.fillna('holi').show(25)

+---------+--------------+-------+---------+
|id number|      position| salary| location|
+---------+--------------+-------+---------+
|        0|     widgeteer|  35000|   Madrid|
|        1|     widgeteer|  35000|Barcelona|
|        2|        wizard|  60000|   Madrid|
|        3|smoke salesman|  50000|Barcelona|
|        4|smoke salesman|  50000|   Madrid|
|        5|smoke salesman|  50000|   Madrid|
|        6|     widgeteer|  35000|Barcelona|
|        7|     widgeteer|  35000|   Madrid|
|        8|     psycopath| 100000|Barcelona|
|        9|     widgeteer|  35000|   Madrid|
|       10|     widgeteer|  35000|   Madrid|
|       11|     widgeteer|  35000|Barcelona|
|       12|smoke salesman|  50000|   Madrid|
|       13|smoke salesman|  50000|Barcelona|
|       14|     widgeteer|  35000|   Madrid|
|       15|smoke salesman|  50000|   Madrid|
|       16|     psycopath| 100000|Barcelona|
|       17|smoke salesman|  50000|Barcelona|
|       18|     psycopath| 100000|   Madrid|
|       19

In [ ]:
# Observar que el salary no se rellena xq no tiene el tipo adecuado

In [229]:
df_union.fillna('casita', subset=['location']).show(25)

+---------+--------------+-------+---------+
|id number|      position| salary| location|
+---------+--------------+-------+---------+
|        0|     widgeteer|  35000|   Madrid|
|        1|     widgeteer|  35000|Barcelona|
|        2|        wizard|  60000|   Madrid|
|        3|smoke salesman|  50000|Barcelona|
|        4|smoke salesman|  50000|   Madrid|
|        5|smoke salesman|  50000|   Madrid|
|        6|     widgeteer|  35000|Barcelona|
|        7|     widgeteer|  35000|   Madrid|
|        8|     psycopath| 100000|Barcelona|
|        9|     widgeteer|  35000|   Madrid|
|       10|     widgeteer|  35000|   Madrid|
|       11|     widgeteer|  35000|Barcelona|
|       12|smoke salesman|  50000|   Madrid|
|       13|smoke salesman|  50000|Barcelona|
|       14|     widgeteer|  35000|   Madrid|
|       15|smoke salesman|  50000|   Madrid|
|       16|     psycopath| 100000|Barcelona|
|       17|smoke salesman|  50000|Barcelona|
|       18|     psycopath| 100000|   Madrid|
|       19

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

In [234]:
session.sql('SELECT * FROM df WHERE location = "Madrid" AND salary > 40000').show()

AnalysisException: 'Table or view not found: df; line 1 pos 14'

In [231]:
df.registerTempTable('df_table')

In [233]:
session.sql('SELECT * FROM df_table WHERE location = "Madrid" AND salary > 40000').show()

+---------+--------------+------+--------+
|id number|      position|salary|location|
+---------+--------------+------+--------+
|        2|        wizard| 60000|  Madrid|
|        4|smoke salesman| 50000|  Madrid|
|        5|smoke salesman| 50000|  Madrid|
|       12|smoke salesman| 50000|  Madrid|
|       15|smoke salesman| 50000|  Madrid|
|       18|     psycopath|100000|  Madrid|
|       19|        wizard| 60000|  Madrid|
+---------+--------------+------+--------+



In [236]:
session.sql('SELECT salary, log1p(salary) FROM df_table WHERE location = "Madrid" AND salary > 40000').show()

+------+-----------------------------+
|salary|LOG1P(CAST(salary AS DOUBLE))|
+------+-----------------------------+
| 60000|           11.002116507732017|
| 50000|           10.819798284210286|
| 50000|           10.819798284210286|
| 50000|           10.819798284210286|
| 50000|           10.819798284210286|
|100000|            11.51293546492023|
| 60000|           11.002116507732017|
+------+-----------------------------+



In [245]:
def classist(salary):
    return 'perro_flauta' if salary < 42000 else 'burgues' 

classist_udf = functions.udf(classist)

In [239]:
session.sql('SELECT salary, classist_udf(salary) FROM df_table WHERE location = "Madrid" AND salary > 40000').show()

AnalysisException: "Undefined function: 'classist_udf'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 15"

In [246]:
session.udf.register('classist_sql', classist_udf)

<function __main__.classist(salary)>

In [247]:
session.sql('SELECT salary, classist_sql(salary) FROM df_table WHERE location = "Madrid" AND salary > 40000').show(25)

+------+--------------------+
|salary|classist_sql(salary)|
+------+--------------------+
| 60000|             burgues|
| 50000|             burgues|
| 50000|             burgues|
| 50000|             burgues|
| 50000|             burgues|
|100000|             burgues|
| 60000|             burgues|
+------+--------------------+



Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

In [250]:
pandas_df = df.toPandas()

pandas_df.head()

,id number,position,salary,location
0,0,widgeteer,35000,Madrid
1,1,widgeteer,35000,Barcelona
2,2,wizard,60000,Madrid
3,3,smoke salesman,50000,Barcelona
4,4,smoke salesman,50000,Madrid


In [251]:
session.createDataFrame(pandas_df)

DataFrame[id number: bigint, position: string, salary: bigint, location: string]

## Writing out


In [252]:
# df.write.csv('df.csv')

# Echar un ojo a cómo lo escribe. Genera un fichero de 0 bytes con sucess y se generan muchisimos ficheros con partes pequeñitas del df 

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

In [287]:
df_coupons = session.read.csv('coupon150720.csv')

In [288]:
df_coupons_mad = df_coupons.where(df_coupons['_c3'] == 'MAD')

In [289]:
df_coupons_mad.show(5)

+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
|79062005639642|  4|BRU|MAD| UX| UX|21.02|USD|  2|  Q|   Q|1172|150912|  OK|INBQ|
|79065668754871|  6|BRU|MAD| SN| 9W|27.66|USD|  2|  W|   W|6021|151018|  OK|INBQ|
|79065668917696|  2|CDG|MAD| AF| AF|46.97|USD|  1|  T|   T|1400|150905|  OK|INB0|
|79062006133090|  2|CDG|MAD| AF| AF| 3.38|USD|  1|  X|   X|1400|151103|  OK|IAE0|
|79062006110497|  1|CDG|MAD| AF| AF|26.02|USD|  1|  Y|   Y|1300|150724|  SA|INA0|
+--------------+---+---+---+---+---+-----+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [290]:
df_agg = df_coupons_mad.groupBy('_c2').agg(functions.sum('_c6').alias('sum_amount'),\
                                 functions.count('_c6').alias('count'))

df_agg.show()

+---+------------------+-----+
|_c2|        sum_amount|count|
+---+------------------+-----+
|PMI|40547.170000000035|  569|
|YUL|            284.44|    1|
|HEL|           8195.76|   74|
|SXB|            264.46|    2|
|UIO|            8547.6|   30|
|XRY| 9250.230000000001|  136|
|OLB|            1801.5|   12|
|CCS|          94528.68|   70|
|VRN|1020.5399999999998|   16|
|SPC| 7542.700000000002|   58|
|AUH|           4393.42|   18|
|JED|19116.159999999996|   30|
|CMN| 7494.280000000001|   77|
|FRA|38863.540000000015|  561|
|ALG|14558.570000000003|  115|
|IST|13363.370000000003|   88|
|SDR| 4219.360000000001|   64|
|TXL|15399.789999999999|  212|
|GRU| 87192.64000000001|  349|
|VIE|          11666.56|  158|
+---+------------------+-----+
only showing top 20 rows



In [291]:
df_agg = df_agg.withColumn('avg', (df_agg['sum_amount'] / df_agg ['count']).cast(types.FloatType()))


In [292]:
df_agg.show(2)

+---+------------------+-----+--------+
|_c2|        sum_amount|count|     avg|
+---+------------------+-----+--------+
|PMI|40547.170000000035|  569|71.26041|
|YUL|            284.44|    1|  284.44|
+---+------------------+-----+--------+
only showing top 2 rows



In [295]:
df_agg.dtypes

[('_c2', 'string'),
 ('sum_amount', 'double'),
 ('count', 'bigint'),
 ('avg', 'float')]

In [300]:
df_agg = df_agg.sort('avg', ascending=False)
df_agg.show(10)

+---+------------------+-----+---------+
|_c2|        sum_amount|count|      avg|
+---+------------------+-----+---------+
|CCS|          94528.68|   70|1350.4097|
|YYZ|3703.1000000000004|    5|   740.62|
|ICN|           8950.84|   13| 688.5262|
|JED|19116.159999999996|   30| 637.2053|
|LAX|17331.340000000004|   29| 597.6324|
|MDE|15621.470000000001|   30|520.71564|
|MCO|             513.5|    1|    513.5|
|MEX|          56316.73|  112|502.82794|
|SCL| 42584.95999999999|   89| 478.4827|
|RUH| 6883.029999999999|   17|404.88412|
+---+------------------+-----+---------+
only showing top 10 rows



1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf